In [7]:
from dash import Dash, html, dash_table, dcc, callback, Output, Input
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import time

import oandapyV20
from oandapyV20 import API
import oandapyV20.endpoints.pricing as pricing
import oandapyV20.endpoints.instruments as instruments

In [8]:
account_id = "ACCOUNT_ID"
token = "TOKEN"
api = oandapyV20.API(access_token=token)

In [9]:
def get_currency_price(instrument_list):
    rst_dict = {}
    for instrument in instrument_list:
        params = {"instruments": instrument, "since": None, "includeUnitsAvailable": False}
        r = pricing.PricingInfo(accountID=account_id, params=params)
        api.request(r)

        price = r.response.get("prices")[0]
        
        rst_dict[instrument] = {}
        rst_dict[instrument]["bid"] = price.get("bids")[0].get("price")
        rst_dict[instrument]["ask"] = price.get("asks")[0].get("price")

    return rst_dict

In [10]:
def get_currency_candle(instrument, granularity = 'S5', count = 30):
    df = pd.DataFrame()
    params = {"granularity": granularity, "count": count}
    r = instruments.InstrumentsCandles(instrument=instrument, params=params)
    api.request(r)

    data = [
        {
            "Time": pd.to_datetime(d["time"].split(".")[0]),
            "High": d["mid"]["h"],
            "Close": d["mid"]["c"],
            "Low": d["mid"]["l"],
            "Open": d["mid"]["o"],
        }
        for d in r.response["candles"]
    ]
    df = pd.DataFrame(data)
    return df

In [11]:
currency_pair_list = ['EUR_USD', 'GBP_USD', 'USD_JPY', 'AUD_USD']

In [12]:
app = Dash(__name__)

app.layout = html.Div([
    html.H1('Forex Trading Dashboard'),
    # show radio button to select the currency pair
    dcc.RadioItems(id='currency-pair', options=[{'label': i, 'value': i} for i in currency_pair_list], value='EUR_USD'),
    # show the candlestick chart of currency price
    dcc.Graph(id='candlestick-chart'),
    # show the latest currency pair bid/ask price
    html.P('Currency Price: '),
    html.Table([
        html.Tr([html.Th('Currency Pair'), html.Th('Bid Price'), html.Th('Ask Price')]),
        html.Tr([html.Td('EUR/USD'), html.Td(id='EURUSD-bid-price'), html.Td(id='EURUSD-ask-price')]),
        html.Tr([html.Td('GBP/USD'), html.Td(id='GBPUSD-bid-price'), html.Td(id='GBPUSD-ask-price')]),
        html.Tr([html.Td('USD/JPY'), html.Td(id='USDJPY-bid-price'), html.Td(id='USDJPY-ask-price')]),
        html.Tr([html.Td('AUD/USD'), html.Td(id='AUDUSD-bid-price'), html.Td(id='AUDUSD-ask-price')])
    ]),
    # update the chart every second
    dcc.Interval(id='interval', interval=5000)
])

@app.callback(
    Output(component_id='candlestick-chart', component_property='figure'),
    Output(component_id='EURUSD-bid-price', component_property='children'),
    Output(component_id='EURUSD-ask-price', component_property='children'),
    Output(component_id='GBPUSD-bid-price', component_property='children'),
    Output(component_id='GBPUSD-ask-price', component_property='children'),
    Output(component_id='USDJPY-bid-price', component_property='children'),
    Output(component_id='USDJPY-ask-price', component_property='children'),
    Output(component_id='AUDUSD-bid-price', component_property='children'),
    Output(component_id='AUDUSD-ask-price', component_property='children'),
    Input(component_id='currency-pair', component_property='value'),
    Input(component_id='interval', component_property='n_intervals')
)
def update_candlestick_chart(currency_pair, n_intervals):
    # get the currency pair selected
    df = get_currency_candle(currency_pair)
    fig = go.Figure(data=[go.Candlestick(x=df['Time'],
                                         open=df['Open'],
                                         high=df['High'],
                                         low=df['Low'],
                                         close=df['Close'])])
    fig.update_layout(title='Candlestick Chart', xaxis_title='Time', yaxis_title='Price', xaxis_rangeslider_visible=False, height=400, width=1300)

    # get the latest currency price
    currency_price_dict = get_currency_price(currency_pair_list)

    return fig, \
        currency_price_dict.get('EUR_USD').get('bid'), currency_price_dict.get('EUR_USD').get('ask'), \
        currency_price_dict.get('GBP_USD').get('bid'), currency_price_dict.get('GBP_USD').get('ask'), \
        currency_price_dict.get('USD_JPY').get('bid'), currency_price_dict.get('USD_JPY').get('ask'), \
        currency_price_dict.get('AUD_USD').get('bid'), currency_price_dict.get('AUD_USD').get('ask')

if __name__ == '__main__':
    app.run_server(jupyter_mode="external", debug=True)

Dash app running on http://127.0.0.1:8050/
